In [79]:
from pearing_class_functions import Pear_Playlists
import time
from pearing_classes import Playlist_Base
import math
#playlist_name_1 = 'Snowboarding Hype'
#playlist_name_2 = 'Snowboarding Chill'
playlist_name_1 = 'Chill Alt Rock'
playlist_name_2 = 'tmp2'

access_token = 'Bearer BQDFouM-d1SBauHUlYuaaznZh8YX4zx5_cQqpF-qSaQTsamaJVqv4xzrQ1zyzSvjPShruEhSomDXZYkW0IqQmxfp9odp3UjuX-C4WAHQwoeWYiOpJHvz9DMErGvyaMfc_gO9m8N4NXxL5TMbrp9fJD19q5THlutWBRsaAJsT2HdAehPEPVNf8I7FG-21TSZKCYp-3sKCBRtg3CGP45Bc0IEmTmOxmVGdkRhHJC_je_Q2PskMyjxRaOpBPVq6-RUoaeo46A'
user_id = '22r6slwbns4u7hkhn3hjhjhyi'


start_time = time.time()



In [80]:
p1_base = Playlist_Base(playlist_name = playlist_name_1, user_id = user_id, access_token = access_token)

Initalized first set:  0.26423001289367676
ran over all playlists:  0.17891478538513184
got tracks from playlist:  0.18864822387695312
Got features of original playlist:  27.433051824569702
got rec tracks:  2.5822060108184814
Got rec track features  102.49434399604797
finished generate_from_playlist_name:  132.87763094902039


In [81]:
from pearing_core_functions import get_tracks_from_playlist
track_df = get_tracks_from_playlist(p1_base.user_id, p1_base.orig_playlist_dict, p1_base.spotipy_connection)


In [129]:
def get_track_features_2(track_df, access_token, user_id):
    cols_to_keep = list(['track_id','danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo'])
    
    # Can only get up to 100 songs at a time from API
    # make empty df, iterate over every 100 songs and append
    track_features_df = pd.DataFrame(columns = cols_to_keep)
    track_list = track_df['track_id'].tolist()
    api_headers = {'Authorization': access_token}
    for i in range(0,(math.ceil(len(track_list)/100))):
        track_list_tmp = track_list[i*100:(i+1)*100]
        api_url = 'https://api.spotify.com/v1/audio-features/?ids=' + ','.join(track_list_tmp)
        api_get_response = requests.get(api_url, headers = api_headers )
        track_features = api_get_response.json()
        track_features_df_tmp = pd.DataFrame(tracks_features_json['audio_features'])
        track_features_df_tmp['track_id'] = track_features_df_tmp['id']
        track_features_df_tmp = track_features_df_tmp[cols_to_keep]
        track_features_df = track_features_df.append(track_features_df_tmp)

    return track_features_df

In [97]:
def get_track_features_old(track_df, spotipy_connection):

    track_df_var = copy.deepcopy(track_df)
    new_cols = list([ 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo'])

    # Create all new columns for feature types
    for col in new_cols:
        # Initialize Column as empty
        track_df_var[col] = np.nan
        # Run over each row
    for i in range(0, len(track_df_var)):
        # Get features for current song
        cur_track_features = spotipy_connection.audio_features(track_df_var['track_id'][[i]])
        #Insert specific feature into column
        #TODO: This currently calls the sp.audio_features like 10 times for each song (since I do it for each row)
        for col in new_cols:
            track_df_var[col].iloc[i] = cur_track_features[0][col]
    
    return track_df_var


In [85]:
old_track_features = get_track_features_old(track_df, p1_base.spotipy_connection)


In [130]:
new_track_features = get_track_features_2(track_df, access_token, user_id)


0


In [131]:
new_track_features

,track_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,3kZC0ZmFWrEHdUCmUqlvgZ,0.517,0.206,5,-12.233,1,0.0421,0.924000,0.000000,0.1180,0.4890,80.401
1,6wNCdMW82LwJgFrnGqLhpJ,0.557,0.859,2,-6.557,1,0.0582,0.004740,0.000058,0.1030,0.7480,154.718
2,59FC22eN2Syt9bbv2d6393,0.608,0.642,2,-7.978,1,0.0251,0.170000,0.066700,0.1660,0.6080,87.703
3,5yc59J3MR3tVDPTOgwgRI5,0.621,0.853,2,-5.735,0,0.0253,0.076700,0.000003,0.0893,0.7140,128.142
4,7DDRPKLKFIvDbNSQmnz19Y,0.439,0.469,9,-9.283,1,0.0311,0.395000,0.573000,0.1830,0.1930,127.440
5,7C1p70Zs3pBZEQGeWMF8sS,0.705,0.856,9,-5.921,1,0.0423,0.041400,0.002280,0.0666,0.7650,117.005
6,7DkTneByAopqOjXFG6XlZK,0.642,0.721,4,-9.765,1,0.0434,0.196000,0.474000,0.0926,0.4380,124.997
7,77vYwoC7e3pVoPq8BA9CuL,0.605,0.619,9,-8.971,0,0.0261,0.423000,0.313000,0.0732,0.5780,133.074
8,2mvBxK1ETGj5nguG6cDaax,0.594,0.630,6,-7.323,0,0.0268,0.043100,0.000003,0.0974,0.3650,117.010
9,1crbFuCkGL4kXnAGd63RXq,0.579,0.662,9,-6.885,1,0.0245,0.000625,0.000004,0.1170,0.5740,102.951
